In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import napari
from scribbles_testing.FoodSeg103_data_handler import *

## Create scribbles

Load the ground truths as batch

In [ ]:
img_nums = [n for n in range(0, 4983, 8)] #[1382] #2750 #1234 #2314
gts = load_food_batch(img_nums, load_images=False)[1]
num_imgs = len(gts)
print(f"Total images: {num_imgs}")

Check and filter for resolution

In [ ]:
resol = {im_num: np.prod(gt.shape) for im_num, gt in gts.items()}

# Check for images with resolution over a certain threshold
thresh = 640_000
print("Resolutions:")
for im_num, r in resol.items(): print(f"{im_num:4d}: {r:9,d} pixels{' (!)' if r > thresh else ''}")

In [ ]:
# Filter out those images
gts_new = {im_num: gt for im_num, gt in gts.items() if resol[im_num] <= thresh}
num_imgs = len(gts_new)
print(f"Images with resolution under {thresh:,d}: {num_imgs}")
resol_new = {im_num: np.prod(gt.shape) for im_num, gt in gts_new.items()}
for im_num, r in resol_new.items(): print(f"{im_num:4d}: {r:7,d} pixels")
gts = gts_new

Define scribbles parameters

In [ ]:
# Which scribbles to create
mode = "all"
class_dist = "balanced"
rel_scribble_len = 10
enforce_max_perc = True
bins = [2.5]#[0.01, 0.025, 0.05, 0.1, 0.25, 0.5, 1, 2.5, 5, 10] #[0.01, 0.03, 0.1, 0.3, 1, 3, 10]#[0.05, 0.25, 1, 5]
scribble_width = 2
# bin_width = [(0.05, 1), (0.1, 1),
            #  (0.25, 2), (0.5, 2), (1, 2),
            #  (2.5, 3), (5, 3),
            #  (10, 4)]
margin = 0.75

# suff = ["TEST"] #["w3"] #standing for width 3
s = "TEST"
scribbles_seed = 1

# Where to save the scribbles
folder_path = "../FoodSeg103_results/data/food_run02"

# What to do
save_res = False
show_res = False
print_steps = True

Loop and create scribbles

In [ ]:
percentages = {}
for bin in bins:
# for bin, scribble_width in bin_width:
    percentages[bin] = []
    for img_num, gt in gts.items():
        print(f"IMG {img_num}: bin {bin}, suff {s}")
        np.random.seed(scribbles_seed)
        scribbles, perc_labelled = create_food_scribble(gt, folder_path, img_num, bin=bin, margin=margin, rel_scribble_len=rel_scribble_len, scribble_width=scribble_width, mode=mode,
                                                        class_dist=class_dist, enforce_max_perc=enforce_max_perc,
                                                        save_res=save_res, suff=s, show_res=show_res, image=None, print_steps=print_steps)
        percentages[bin].append(perc_labelled)
        print("\n")

Quick report of labelling percentages:

In [ ]:
for bin, perc_list in percentages.items():
    print(f"labelled percentage in bin {bin}: mean = {np.mean(perc_list):.4f}, std = {np.std(perc_list):.4f}, range = {np.min(perc_list):.4f} - {np.max(perc_list):.4f}")

Check percentages of saved scribbles

In [ ]:
perc_dict = {}
for bin in bins:
    bin_perc_list = []
    for img_num in range(0,540,20):
        scrib = np.array(Image.open(f"{folder_path}{img_num:03d}_scribbles_{mode}_{bin_for_file(bin)}_{s}.png"))
        perc_labelled = np.sum(scrib > 0) / scrib.size
        bin_perc_list.append(perc_labelled)
    perc_dict[bin] = bin_perc_list

Show the last created scribble with its own ground truth, and (optionally) all other ground truths

In [ ]:
v = napari.Viewer()
v.add_labels(gt, name="Ground Truth")
v.add_labels(scribbles, name="Scribbles")

In [ ]:
for g in gts:
    v.add_labels(gts[g], name=f"GT {g}")